In [1]:
from mxnet import autograd, nd
from mxnet.gluon import nn

In [19]:
def corr2d(X, K):
    h, w = K.shape
    Y = nd.zeros((X.shape[0] - h +1, X.shape[1] - w +1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

In [23]:
X = nd.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = nd.random.uniform(shape=(1,2))
with autograd.record():
    l = corr2d(X, K)
l.backward()

MXNetError: [21:02:48] src/imperative/imperative.cc:295: Check failed: !AGInfo::IsNone(*i): Cannot differentiate node because it is not in a computational graph. You need to set is_recording to true or use autograd.record() to save computational graphs for backward. If you want to differentiate the same graph twice, you need to pass retain_graph=True to backward.
Stack trace:
  [bt] (0) 1   libmxnet.so                         0x000000010fe44929 mxnet::op::NDArrayOpProp::~NDArrayOpProp() + 4473
  [bt] (1) 2   libmxnet.so                         0x00000001113cbea1 mxnet::Imperative::Backward(std::__1::vector<mxnet::NDArray*, std::__1::allocator<mxnet::NDArray*> > const&, std::__1::vector<mxnet::NDArray*, std::__1::allocator<mxnet::NDArray*> > const&, std::__1::vector<mxnet::NDArray*, std::__1::allocator<mxnet::NDArray*> > const&, bool, bool, bool) + 16353
  [bt] (2) 3   libmxnet.so                         0x0000000111308ffe MXAutogradBackwardEx + 1022
  [bt] (3) 4   libffi.6.dylib       

In [11]:
class Conv2D(nn.Block):
    def __init__(self, kernel_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        self.weight = slef.params.get('weight', shape=kernel_size)
        self.bias = self.params.get('bias', shape=(1,))
    
    def forward(self, x):
        return corr2d(x, self.weight.data()) + self.bias.data() 

In [12]:
X = nd.ones((6, 8))
X[:, 2:6] = 0
X


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

In [13]:
K = nd.array([[1, -1]])
Y = corr2d(X, K)
Y


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

In [15]:
conv2d = nn.Conv2D(1, kernel_size=(1, 2))
conv2d.initialize()

In [17]:
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    with autograd.record():
        Y_hat = conv2d(X)
        l = (Y_hat - Y) ** 2
    l.backward()
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
    if (i + 1) % 2 == 0:
        print('batch %d, loss %.3f' % (i + 1, l.sum().asscalar()))

batch 2, loss 0.831
batch 4, loss 0.140
batch 6, loss 0.024
batch 8, loss 0.004
batch 10, loss 0.001


In [18]:
conv2d.weight.data().reshape((1, 2))


[[ 0.9959445  -0.99458164]]
<NDArray 1x2 @cpu(0)>